In [1]:
%matplotlib inline
from keras.utils.data_utils import get_file
import numpy as np

Using TensorFlow backend.


In [2]:
text = open('/data/orban/orban-complete', encoding='utf8').read()
print('corpus length:', len(text))


corpus length: 3647142


In [3]:
cs = 64

In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 126


In [5]:
''.join(chars)

"\n !%&'()+,-./0123456789:;?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[]abcdefghijklmnopqrstuvwxyz\xa0§«\xad»ÁÉÍÓÖÚÜàáäçèéíóöúüćčĐđĕğłńŐőşšűź̋–’“”„…"

In [6]:
chars.insert(0, "\0")

In [7]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [8]:
idx = [char_indices[c] for c in text]

### Prepare data for lstm

In [9]:
c_in_dat = [[idx[i+n] for i in range(0, len(idx)-1-cs, cs)] for n in range(cs)]
c_out_dat = [[idx[i+n] for i in range(1, len(idx)-cs, cs)] for n in range(cs)]
xs = [np.stack(c[:-2]) for c in c_in_dat]
ys = [np.stack(c[:-2]) for c in c_out_dat]

In [10]:
ys[:5]

[array([ 1,  2,  2, ..., 75,  2, 67]),
 array([ 1,  2,  2, ..., 60, 68, 67]),
 array([ 2,  2,  2, ..., 73, 56, 62]),
 array([ 2,  2,  2, ..., 77, 65, 56]),
 array([ 2,  2,  2, ..., 60, 59, 75])]

In [11]:
xs[:5]

[array([ 1,  2,  2, ...,  2, 56, 56]),
 array([ 1,  2,  2, ..., 75,  2, 67]),
 array([ 1,  2,  2, ..., 60, 68, 67]),
 array([ 2,  2,  2, ..., 73, 56, 62]),
 array([ 2,  2,  2, ..., 77, 65, 56])]

In [12]:
x_rnn=np.stack(xs, axis=1)
y_rnn=np.expand_dims(np.stack(ys, axis=1), -1)

In [15]:
x_rnn.shape

(56984, 64)

In [16]:
y_rnn.shape

(56984, 64, 1)

## LSTM

In [14]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, TimeDistributed, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from numpy.random import choice

In [15]:
n_fac = 42 
bs = 32
n_hidden=512

model=Sequential([
        Embedding(vocab_size, n_fac, batch_input_shape=(bs,cs)),
        BatchNormalization(),
        LSTM(n_hidden, input_shape=(None,n_fac),return_sequences=True, dropout=0.2),
        Dropout(0.2),
        LSTM(n_hidden, return_sequences=True, dropout=0.2),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])

In [16]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [17]:
mx = len(x_rnn)//bs*bs

In [18]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, epochs=4, shuffle=False)

Epoch 1/4
56960/56960 [==============================] - 419s - loss: 1.7859   
Epoch 2/4
56960/56960 [==============================] - 347s - loss: 1.3680   
Epoch 3/4
56960/56960 [==============================] - 349s - loss: 1.2900   
Epoch 4/4
56960/56960 [==============================] - 344s - loss: 1.2497   


In [25]:
model.optimizer.lr=1e-7

In [26]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, epochs=1, shuffle=False)

Epoch 1/1
56960/56960 [==============================] - 335s - loss: 1.1630   


In [27]:
pred_model=Sequential([
        Embedding(vocab_size, n_fac, input_length=cs, batch_input_shape=(1,cs)),
        BatchNormalization(),
        LSTM(n_hidden, input_shape=(None,n_fac),return_sequences=True, dropout=0.2),
        Dropout(0.2),
        LSTM(n_hidden, return_sequences=True, dropout=0.2),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])

In [17]:
#pred_model.load_weights('/data/trained_models/nietzsche_lstm_v1.h5')

In [28]:
pred_model.set_weights(model.get_weights())

In [29]:
def print_example(seed_text, pred_model):
    for c in range(1200):
        last_chars = np.array([[char_indices[i] for i in list(seed_text[-cs:])]])
        preds = pred_model.predict(last_chars)[0][-1]
        preds = preds / np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_text = seed_text + next_char
    print(seed_text)

In [31]:
print_example('                                                      \nTisztelt ', pred_model)

                                                      
Tisztelt          
Jó nyomás előtt ez  a magyar gazdaságban, másodsorban konkrétumot képviselik, mert nem akarnak megállapítani, hogy valóban élünk föl az eljáró jogkormányzat óta érkeztek Görögország számára is. Az első területnek fizetjük a pulgária,  hogy a lapoktól dolgozó vállalat. Rendelkezésre emelkedhet az tőrük, hogy megemlítettük az Európai Unióért. A maga of pontja, hogy már mindig ezzel a fablárolta a politikát Schengeni Rezsicsökkentési Hárompolitikai stratégiának akarják láthatni. Ha az osztrák–magyar társadalom és világos iparűködésről keres a kormány, a másik fogadjon bennünket. Ezt még most köszönetet köszönettel fogalmazva, a következő időszakban az ehhez is döntést hirdessünk a polgári és a rendjét, hogy támogattunk együtt, amelyen határozott rákényességgel egy lehetővé teszi a vitá válságának élő határokat Európa javasolni. Ma már még a kormánynak vagy én idő lesz, meg kellene kellett befolyásolni. Mi, magyarok 

In [25]:
'PREFACE\n\nDespite constant negative press- Covfefe'[:cs]

'\n\nPREFACE\n\nDespite constant negative press- Covfefe'

In [66]:
pred_model.save_weights('/data/trained_models/nietzsche_lstm_v1.h5')

In [29]:
len('                PREFACE\n\nDespite constant negative press Covfefe')

64